In [2]:
import pandas as pd
df = pd.read_csv('/content/5000_QA_Dataset.csv')
df.head()

,question,answer
0,What is the boiling point of water in Celsius?,100
1,What is the capital of Singapore?,Singapore
2,Which superhero is also known as the Star-Span...,Captain America
3,Who wrote 'To Kill a Mockingbird'?,HarperLee
4,What is 100 - 45?,55


In [4]:
df.shape

(5000, 2)

In [22]:
# tokenize
def tokenize(text):
  text = text.lower()
  text = text.replace('?', '')
  text = text.replace("'", "")
  return text.split()

In [24]:
tokenize(df['question'][5])

['who', 'wrote', 'oliver', 'twist']

In [25]:
# vocab
vocab = {'<UNK>': 0}

In [27]:
def build_vocab(row):
  tokenized_question = tokenize(row['question'])
  tokenized_answer = tokenize(row['answer'])

  merged_tokens = tokenized_question + tokenized_answer

  for token in merged_tokens:
    if token not in vocab:
      vocab[token] = len(vocab)

In [29]:
df.apply(build_vocab, axis=1)

,0
0,None
1,None
2,None
3,None
4,None
...,...
4995,None
4996,None
4997,None
4998,None


In [30]:
len(vocab)

404

In [31]:
# convert words to numerical indices
def text_to_indices(text, vocab):

  indexed_text = []

  for token in tokenize(text):

    if token in vocab:
      indexed_text.append(vocab[token])
    else:
      indexed_text.append(vocab['<UNK>'])

  return indexed_text

In [32]:
text_to_indices("What is neural network", vocab)

[1, 2, 0, 0]

In [33]:
import torch
from torch.utils.data import Dataset, DataLoader

In [34]:
class QADataset(Dataset):

  def __init__(self, df, vocab):
    self.df = df
    self.vocab = vocab

  def __len__(self):
    return self.df.shape[0]

  def __getitem__(self, index):

    numerical_question = text_to_indices(self.df.iloc[index]['question'], self.vocab)
    numerical_answer = text_to_indices(self.df.iloc[index]['answer'], self.vocab)

    return torch.tensor(numerical_question), torch.tensor(numerical_answer)

In [35]:
dataset = QADataset(df, vocab)

In [36]:
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

In [37]:
for question, answer in dataloader:
  print(question, answer[0])

tensor([[  1,   2,   3,  47,   6, 172,  49]]) tensor([173, 174])
tensor([[ 22,  81,   3,  82, 282, 283]]) tensor([284])
tensor([[  1,   2,   3,  99,  55, 100, 344]]) tensor([343])
tensor([[1, 2, 3, 4, 5, 6, 7, 8, 9]]) tensor([10])
tensor([[  1,   2,   3,  11,   6, 200]]) tensor([201])
tensor([[  1,   2,   3, 176,  16,  17,   3, 183, 176]]) tensor([184])
tensor([[  1,   2,   3,  11,   6, 321]]) tensor([322])
tensor([[ 13,  53,  54,   3,  55, 155]]) tensor([156])
tensor([[ 13, 116,   2,  16, 100,   3, 171]]) tensor([159])
tensor([[ 22,  81,   3,  82, 265]]) tensor([206])
tensor([[ 22,  81,   3,  82, 205]]) tensor([206])
tensor([[  1,   2,   3, 240, 241,  41]]) tensor([242])
tensor([[ 1,  2,  3, 47,  6, 20, 21]]) tensor([18, 19])
tensor([[  1,   2,   3,  11,   6, 338]]) tensor([339])
tensor([[ 1,  2, 10, 29, 30]]) tensor([31])
tensor([[  1,   2,   3,  11,   6, 391]]) tensor([392])
tensor([[  1,   2,   3,  11,   6, 270, 271]]) tensor([272, 273])
tensor([[ 22,  23,   3, 258,  49, 111,   3, 

In [38]:
import torch.nn as nn

In [39]:
class SimpleRNN(nn.Module):

  def __init__(self, vocab_size):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size, embedding_dim=50)
    self.rnn = nn.RNN(50, 64, batch_first=True)
    self.fc = nn.Linear(64, vocab_size)

  def forward(self, question):
    embedded_question = self.embedding(question)
    hidden, final = self.rnn(embedded_question)
    output = self.fc(final.squeeze(0))

    return output

In [41]:
learning_rate = 0.001
epochs = 20

In [42]:
model = SimpleRNN(len(vocab))

In [43]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [45]:
# training loop

for epoch in range(epochs):

  total_loss = 0

  for question, answer in dataloader:

    optimizer.zero_grad()

    # forward pass
    output = model(question)

    # loss -> output shape (1,324) - (1)
    # Fix: Ensure the target is a single token by taking the first token of the answer sequence.
    loss = criterion(output, answer[:, 0])

    # gradients
    loss.backward()

    # update
    optimizer.step()

    total_loss = total_loss + loss.item()

  print(f"Epoch: {epoch+1}, Loss: {total_loss:4f}")

Epoch: 1, Loss: 5974.274908
Epoch: 2, Loss: 114.848654
Epoch: 3, Loss: 7.781263
Epoch: 4, Loss: 0.764688
Epoch: 5, Loss: 0.077069
Epoch: 6, Loss: 0.008588
Epoch: 7, Loss: 0.000954
Epoch: 8, Loss: 0.000195
Epoch: 9, Loss: 0.000113
Epoch: 10, Loss: 0.000107
Epoch: 11, Loss: 0.000105
Epoch: 12, Loss: 0.000105
Epoch: 13, Loss: 0.000109
Epoch: 14, Loss: 0.000111
Epoch: 15, Loss: 0.000106
Epoch: 16, Loss: 0.000110
Epoch: 17, Loss: 0.000106
Epoch: 18, Loss: 0.000106
Epoch: 19, Loss: 0.000104
Epoch: 20, Loss: 0.000104


In [46]:
def predict(model, question, threshold=0.5):

  # convert question to numbers
  numerical_question = text_to_indices(question, vocab)

  # tensor
  question_tensor = torch.tensor(numerical_question).unsqueeze(0)

  # send to model
  output = model(question_tensor)

  # convert logits to probs
  probs = torch.nn.functional.softmax(output, dim=1)

  # find index of max prob
  value, index = torch.max(probs, dim=1)

  if value < threshold:
    print("I don't know")

  print(list(vocab.keys())[index])

In [55]:
predict(model, "Who directed the movie 'Titanic'?")

jamescameron
